In [1]:
var('t q')

def L_partition_vector2(partition, vector):
    for i in range(len(partition)):
        vector = d[-partition[len(partition) - i - 1]]*vector
    return vector

def my_coefficient(partition, vector):
    return vector.monomial_coefficients().get(tuple(-k for k in partition), 0)

def series_t_q(list_of_list_of_partition):
    series = 0
    for i in range(len(list_of_list_of_partition)):
        series_t = 0
        for j in range(len(list_of_list_of_partition[i])):
            series_t = series_t + t^len(list_of_list_of_partition[i][j])
        series = series + series_t*q^i
    return series

def partition_ends(partition, tail, is_greater):
    if len(partition) < len(tail) - 1:
        return False
    partition2 = partition[len(partition) - len(tail) + 1: len(partition)]
    if partition2 != tail[1:]:
        return False
    if is_greater:
        if partition == partition2:
            return True
        if partition[len(partition) - len(tail)] > tail[0]:
            return True
    else:
        if partition == partition2:
            return False
        if partition[len(partition) - len(tail)] == tail[0]:
            return True
    return False

def list_of_list_of_partition_ends(list_of_list_of_partition, tail, is_greater):
    return [[partition
             for partition in list_of_partition if partition_ends(partition, tail, is_greater)]
            for list_of_partition in list_of_list_of_partition]

def my_order(partition):
    polynomial = Partition(partition).to_exp(sum(partition))
    polynomial = [-k for k in polynomial]
    polynomial.reverse()
    return ((len(partition), polynomial)) 

def ordered_partitions(N):
    if N <= 0:
        return [[]]
    list_of_partition = list(Partitions(N))
    list_of_partition.sort(key = my_order, reverse = True)
    return list_of_partition

def find_singular_vector_at_weight(weight):
    list_of_partition = ordered_partitions(weight)
    list_of_partition.reverse()
    list_of_partition2 = ordered_partitions(weight - 1)
    list_of_partition2.reverse() 
    list_of_partition3 = ordered_partitions(weight - 2)
    list_of_partition3.reverse()
    matrix_1 = matrix({(i, j): my_coefficient(list_of_partition2[i], d[1]*L_partition_vector2(list_of_partition[j], v)) for j in range(len(list_of_partition)) for i in range(len(list_of_partition2))}, sparse = False)
    matrix_2 = matrix({(i, j): my_coefficient(list_of_partition3[i], d[2]*L_partition_vector2(list_of_partition[j], v)) for j in range(len(list_of_partition)) for i in range(len(list_of_partition3))}, sparse = False)
    matrix_1 = matrix_1.stack(matrix_2)
    matrix_1 = matrix_1.rref()
    vector = [-matrix_1[k, len(list_of_partition)-1] for k in range(len(list_of_partition)-1)]
    vector_list = list(reversed(vector + [1]))
    denominators = [value.denominator() for value in vector_list]
    common_denominator = LCM(denominators)
    vector_list = [value*common_denominator for value in vector_list]
    list_of_partition.reverse()
    vector2 = {tuple(list_of_partition[i]): vector_list[i] for i in range(len(list_of_partition))}
    return vector2

def find_singular_vectors():
    return (find_singular_vector_at_weight(m*n), find_singular_vector_at_weight((pp - m)*(qq - n)))

def vector_sum(vector1, vector2):
    return {partition: vector1.get(partition, 0) + vector2.get(partition, 0)
            for partition in set(vector1)|set(vector2)}

def vector_simplify0(partition):
    for i in range(len(partition) - 1):
        if partition[i] < partition[i + 1]:
            return {partition[0: i] + (partition[i] + partition[i + 1],)
                    + partition[i + 2: len(partition)]: partition[i + 1] - partition[i],
                    partition[0: i] + (partition[i + 1],) + (partition[i],)
                    + partition[i + 2: len(partition)]: 1}
    return {partition: 1}

def vector_simplify(vector):
    vector2 = {}
    for partition in set(vector):
        vector3 = vector_simplify0(partition)
        vector2 = vector_sum(vector2, {partition2: vector3[partition2]*vector[partition]
                                       for partition2 in set(vector3)})
    if vector2 == vector:
        return vector2
    return vector_simplify(vector2)

def L_partition_vector(partition, vector):
    return vector_simplify(
        {partition + partition2: vector[partition2] for partition2 in set(vector)})

def matrix_UVir_m1_m2(N):
    singular_vectors = find_singular_vectors()
    m1 = singular_vectors[0]
    m2 = singular_vectors[1]
    list_of_partition = ordered_partitions(N)
    list_of_partition2 = ordered_partitions(N - m*n)
    list_of_partition3 = ordered_partitions(N - (pp - m)*(qq - n))
    list_of_vector_m1 = [L_partition_vector(tuple(list_of_partition2[i]), m1)
                         for i in range(len(list_of_partition2))]
    list_of_vector_m2 = [L_partition_vector(tuple(list_of_partition3[i]), m2)
                         for i in range(len(list_of_partition3))]
    matrix_m1 = matrix({(i, j): list_of_vector_m1[i].get(tuple(list_of_partition[j]), 0)
                        for j in range(len(list_of_partition))
                        for i in range(len(list_of_partition2))}, sparse = False)
    matrix_m2 = matrix({(i, j): list_of_vector_m2[i].get(tuple(list_of_partition[j]), 0)
                        for j in range(len(list_of_partition))
                        for i in range(len(list_of_partition3))}, sparse = False)
    matrix_m1 = matrix_m1.stack(matrix_m2)
    matrix_m1 = matrix_m1.rref()
    return matrix_m1

def pivots_up_to(N):
    list_of_list_of_partition = []
    for i in range(N + 1):
        list_of_partition = ordered_partitions(i)
        list_of_list_of_partition.append(
            [list_of_partition[j] for j in matrix_UVir_m1_m2(i).pivots()])
    return list_of_list_of_partition

def basis_up_to(N):
    list_of_list_of_partition = []
    pivots = pivots_up_to(N)
    for i in range(N + 1):
        list_of_partition = ordered_partitions(i)
        list_of_list_of_partition.append(
            [partition for partition in list_of_partition if partition not in pivots[i]])
    return list_of_list_of_partition

def is_subpartition(partition, partition2):
    if len(partition) < len(partition2):
        return False
    for i in range(len(partition) - len(partition2) + 1):
        if partition[i: i + len(partition2)] == partition2:
            return True
    return False

def partition_contains_any_of_list(partition, list_of_partition):
    for partition2 in list_of_partition:
        if is_subpartition(partition, partition2):
            return True
    return False

def partition_contains_any_of_R(partition, s, list_of_partition):
    if partition == []:
        return False
    if partition_contains_any_of_list(partition, list_of_partition):
        return True
    if len(partition) < s:
        return False
    for i in range(len(partition) - s + 1):
        if partition[i] - partition[i + s - 1] < 2:
            return True
    return False

def basis_up_tov2(N, s, list_of_partition):
    list_of_list_of_partition = []
    for i in range(N + 1):
        list_of_partition2 = ordered_partitions(i)
        list_of_list_of_partition.append(
            [partition for partition in list_of_partition2
             if not partition_contains_any_of_R(partition, s, list_of_partition)])
    return list_of_list_of_partition

In [2]:
ordered_partitions(6)

[[1, 1, 1, 1, 1, 1],
 [2, 1, 1, 1, 1],
 [2, 2, 1, 1],
 [3, 1, 1, 1],
 [2, 2, 2],
 [3, 2, 1],
 [4, 1, 1],
 [3, 3],
 [4, 2],
 [5, 1],
 [6]]

In [3]:
Vir = lie_algebras.VirasoroAlgebra(QQ)
pp = 2
ss = 3
m = 1
n = 3
NN = 20
qq = 2*ss + 1
c = 1 - 6*(pp - qq)^2/(pp*qq)
h = ((n*pp - m*qq)^2 - (pp - qq)^2)/(4*pp*qq)
M = Vir.verma_module(c, h)
d = Vir.basis()
v = M.highest_weight_vector()

In [4]:
(c, h)

(-68/7, -3/7)

In [5]:
%time basis_up_to(NN) == basis_up_tov2(NN, ss, [[1, 1, 1]])

CPU times: user 13.5 s, sys: 86.3 ms, total: 13.6 s
Wall time: 14.7 s


True

In [6]:
%time w = basis_up_tov2(40, ss, [[1, 1, 1]])

CPU times: user 3.4 s, sys: 73.8 ms, total: 3.48 s
Wall time: 3.77 s


In [7]:
series_t_q(w)

(55*t^8 + 373*t^7 + 804*t^6 + 808*t^5 + 442*t^4 + 132*t^3 + 20*t^2 + t)*q^40 + (36*t^8 + 298*t^7 + 685*t^6 + 724*t^5 + 406*t^4 + 126*t^3 + 19*t^2 + t)*q^39 + (25*t^8 + 234*t^7 + 586*t^6 + 645*t^5 + 377*t^4 + 119*t^3 + 19*t^2 + t)*q^38 + (15*t^8 + 183*t^7 + 492*t^6 + 574*t^5 + 345*t^4 + 113*t^3 + 18*t^2 + t)*q^37 + (10*t^8 + 139*t^7 + 417*t^6 + 507*t^5 + 319*t^4 + 107*t^3 + 18*t^2 + t)*q^36 + (5*t^8 + 106*t^7 + 344*t^6 + 448*t^5 + 290*t^4 + 101*t^3 + 17*t^2 + t)*q^35 + (3*t^8 + 78*t^7 + 287*t^6 + 392*t^5 + 267*t^4 + 95*t^3 + 17*t^2 + t)*q^34 + (t^8 + 57*t^7 + 233*t^6 + 343*t^5 + 241*t^4 + 90*t^3 + 16*t^2 + t)*q^33 + (t^8 + 40*t^7 + 191*t^6 + 297*t^5 + 221*t^4 + 84*t^3 + 16*t^2 + t)*q^32 + (28*t^7 + 151*t^6 + 257*t^5 + 198*t^4 + 79*t^3 + 15*t^2 + t)*q^31 + (18*t^7 + 122*t^6 + 220*t^5 + 180*t^4 + 74*t^3 + 15*t^2 + t)*q^30 + (12*t^7 + 93*t^6 + 188*t^5 + 160*t^4 + 69*t^3 + 14*t^2 + t)*q^29 + (7*t^7 + 73*t^6 + 158*t^5 + 145*t^4 + 64*t^3 + 14*t^2 + t)*q^28 + (4*t^7 + 54*t^6 + 133*t^5 + 127*t^

In [8]:
series_t_q(list_of_list_of_partition_ends(w, [1], True))

(t^8 + 57*t^7 + 287*t^6 + 448*t^5 + 319*t^4 + 113*t^3 + 19*t^2 + t)*q^40 + (40*t^7 + 233*t^6 + 392*t^5 + 290*t^4 + 107*t^3 + 18*t^2 + t)*q^39 + (28*t^7 + 191*t^6 + 343*t^5 + 267*t^4 + 101*t^3 + 18*t^2 + t)*q^38 + (18*t^7 + 151*t^6 + 297*t^5 + 241*t^4 + 95*t^3 + 17*t^2 + t)*q^37 + (12*t^7 + 122*t^6 + 257*t^5 + 221*t^4 + 90*t^3 + 17*t^2 + t)*q^36 + (7*t^7 + 93*t^6 + 220*t^5 + 198*t^4 + 84*t^3 + 16*t^2 + t)*q^35 + (4*t^7 + 73*t^6 + 188*t^5 + 180*t^4 + 79*t^3 + 16*t^2 + t)*q^34 + (2*t^7 + 54*t^6 + 158*t^5 + 160*t^4 + 74*t^3 + 15*t^2 + t)*q^33 + (t^7 + 41*t^6 + 133*t^5 + 145*t^4 + 69*t^3 + 15*t^2 + t)*q^32 + (28*t^6 + 110*t^5 + 127*t^4 + 64*t^3 + 14*t^2 + t)*q^31 + (21*t^6 + 91*t^5 + 114*t^4 + 60*t^3 + 14*t^2 + t)*q^30 + (13*t^6 + 73*t^5 + 99*t^4 + 55*t^3 + 13*t^2 + t)*q^29 + (9*t^6 + 59*t^5 + 88*t^4 + 51*t^3 + 13*t^2 + t)*q^28 + (5*t^6 + 46*t^5 + 75*t^4 + 47*t^3 + 12*t^2 + t)*q^27 + (3*t^6 + 36*t^5 + 66*t^4 + 43*t^3 + 12*t^2 + t)*q^26 + (t^6 + 27*t^5 + 55*t^4 + 39*t^3 + 11*t^2 + t)*q^25 + 

In [9]:
series_t_q(list_of_list_of_partition_ends(w, [1, 1], True))

(13*t^8 + 158*t^7 + 337*t^6 + 276*t^5 + 106*t^4 + 18*t^3 + t^2)*q^40 + (8*t^8 + 125*t^7 + 292*t^6 + 253*t^5 + 100*t^4 + 18*t^3 + t^2)*q^39 + (4*t^8 + 96*t^7 + 250*t^6 + 228*t^5 + 94*t^4 + 17*t^3 + t^2)*q^38 + (2*t^8 + 74*t^7 + 214*t^6 + 208*t^5 + 89*t^4 + 17*t^3 + t^2)*q^37 + (t^8 + 54*t^7 + 181*t^6 + 186*t^5 + 83*t^4 + 16*t^3 + t^2)*q^36 + (40*t^7 + 152*t^6 + 168*t^5 + 78*t^4 + 16*t^3 + t^2)*q^35 + (28*t^7 + 126*t^6 + 149*t^5 + 73*t^4 + 15*t^3 + t^2)*q^34 + (19*t^7 + 104*t^6 + 134*t^5 + 68*t^4 + 15*t^3 + t^2)*q^33 + (12*t^7 + 84*t^6 + 117*t^5 + 63*t^4 + 14*t^3 + t^2)*q^32 + (8*t^7 + 68*t^6 + 104*t^5 + 59*t^4 + 14*t^3 + t^2)*q^31 + (4*t^7 + 53*t^6 + 90*t^5 + 54*t^4 + 13*t^3 + t^2)*q^30 + (2*t^7 + 41*t^6 + 79*t^5 + 50*t^4 + 13*t^3 + t^2)*q^29 + (t^7 + 31*t^6 + 67*t^5 + 46*t^4 + 12*t^3 + t^2)*q^28 + (23*t^6 + 58*t^5 + 42*t^4 + 12*t^3 + t^2)*q^27 + (16*t^6 + 48*t^5 + 38*t^4 + 11*t^3 + t^2)*q^26 + (11*t^6 + 41*t^5 + 35*t^4 + 11*t^3 + t^2)*q^25 + (7*t^6 + 33*t^5 + 31*t^4 + 10*t^3 + t^2)*q^2

In [10]:
series_t_q(list_of_list_of_partition_ends(w, [1, 1, 1], True))

(41*t^8 + 158*t^7 + 180*t^6 + 84*t^5 + 17*t^4 + t^3)*q^40 + (28*t^8 + 133*t^7 + 160*t^6 + 79*t^5 + 16*t^4 + t^3)*q^39 + (21*t^8 + 110*t^7 + 145*t^6 + 74*t^5 + 16*t^4 + t^3)*q^38 + (13*t^8 + 91*t^7 + 127*t^6 + 69*t^5 + 15*t^4 + t^3)*q^37 + (9*t^8 + 73*t^7 + 114*t^6 + 64*t^5 + 15*t^4 + t^3)*q^36 + (5*t^8 + 59*t^7 + 99*t^6 + 60*t^5 + 14*t^4 + t^3)*q^35 + (3*t^8 + 46*t^7 + 88*t^6 + 55*t^5 + 14*t^4 + t^3)*q^34 + (t^8 + 36*t^7 + 75*t^6 + 51*t^5 + 13*t^4 + t^3)*q^33 + (t^8 + 27*t^7 + 66*t^6 + 47*t^5 + 13*t^4 + t^3)*q^32 + (20*t^7 + 55*t^6 + 43*t^5 + 12*t^4 + t^3)*q^31 + (14*t^7 + 48*t^6 + 39*t^5 + 12*t^4 + t^3)*q^30 + (10*t^7 + 39*t^6 + 36*t^5 + 11*t^4 + t^3)*q^29 + (6*t^7 + 33*t^6 + 32*t^5 + 11*t^4 + t^3)*q^28 + (4*t^7 + 26*t^6 + 29*t^5 + 10*t^4 + t^3)*q^27 + (2*t^7 + 22*t^6 + 26*t^5 + 10*t^4 + t^3)*q^26 + (t^7 + 16*t^6 + 23*t^5 + 9*t^4 + t^3)*q^25 + (13*t^6 + 20*t^5 + 9*t^4 + t^3)*q^24 + (9*t^6 + 18*t^5 + 8*t^4 + t^3)*q^23 + (7*t^6 + 15*t^5 + 8*t^4 + t^3)*q^22 + (4*t^6 + 13*t^5 + 7*t^4 + t^